In [1]:
import pandas as pd
import gurobipy as grb
import math

participants = pd.read_csv("Week 2 Semester 2 Internals Sign Ups! (Responses) - Form Responses 1.csv",index_col=0)

In [11]:
def generatefts(allowPros,allowAllNov):
    global pros
    pros = {}
    global novices
    novices = {}
    strengths = {}
    global allspeakers
    allspeakers = []

    proimportindex = 1
    noviceimportindex = 1
    masterindex = 1
    for ind,row in participants.iterrows():
        if row[3] == "Weak" or row[3] == "Strong":
            pros[proimportindex] = row[1]
            proimportindex += 1
            allspeakers.append(row[1])
        elif row[4] == "Weak" or row[4] == "Strong":
            novices[noviceimportindex] = row[1]
            noviceimportindex += 1
            strengths[row[1]] = row[4]
            allspeakers.append(row[1])

    if len(allspeakers) % 2 > 0:
        print("***Odd number of speakers - Pro Swing added***")
        pros[proimportindex] = "Pro Swing"
        proimportindex += 1
        allspeakers.append("Pro Swing")
    else:
        print("Even number of speakers - OK to proceed")


    #if len(pros) < len(novices)/2:
    #    print("{} Pros, {} Novices - Insufficient pros to proceed with allocation.".format(len(pros),len(novices)))
    #else:
    #    print("{} Pros, {} Novices - Proceed with allocation.".format(len(pros),len(novices)))
    
    
    isAllNov = {}
    
    global fts
    fts = {}
    ftindex = 1
    if allowPros:
        for key,pro in pros.iteritems():
            for key1,novice1 in novices.iteritems():
                fts[ftindex] = pro,novice1
                ftindex += 1
    
    if allowAllNov:
        for key1,novice1 in novices.iteritems():
            for key2,novice2 in novices.iteritems():
                    if novice1 != novice2:
                        fts[ftindex] = novice1,novice2
                        ftindex +=1


    print("{} possible team configurations".format(len(fts)))
    global costs
    costs = {}
    
    for key in fts.keys():
        costs[key] = 0

    for key,team in fts.iteritems():
        novRanks = []
        prosInTeam = 0
        for speaker in team:
            try:
                novRanks.append(strengths[speaker])
            except:
                prosInTeam += 1
    
        if prosInTeam == 0:
            
            strongs = 0
            weaks = 0
            
            for i in (0,1):
                if novRanks[i] == "Strong":
                    strongs += 1
                elif novRanks[i] == "Weak":
                    weaks += 1
            
            # Prefer less concentration of strong novices (more spread across teams)
            if strongs == 2:
                costs[key] = 50
                isAllNov[key] = "2 Strong"
            elif strongs == 1:
                costs[key] == 100
                isAllNov[key] = "1 Strong"

            # Never allow two weak novices to go together
            elif strongs == 0:
                costs[key] = -100
                isAllNov[key] = "0 Strong"
        
        else:
            # Preference pro-teams to have weaker novice where allNov teams are allowed
            if novRanks[0] == "Strong":
                costs[key] = 50
            elif novRanks[0] == "Weak":
                costs[key] = 100
            else:
                costs[key] = 1
                
    for key,value in costs.iteritems():
        x[key] = m.addVar(vtype = grb.GRB.BINARY, obj = value, name="x_{}".format(key))

def pair(speaker1,speaker2):
    if speaker1 not in allspeakers:
        print("No such speaker named {}".format(speaker1))
    elif speaker2 not in allspeakers:
        print("No such speaker named {}".format(speaker2))
    else:
        affected = []
        for key,team in fts.iteritems():
            if speaker1 in team and speaker2 in team:
                affected.append(key)
        #print(affected)
        m.addConstr(sum(x[team] for team in affected) == 1)
        
    
def allocate():
    # Each person must be on exactly one team
    for key,person in pros.iteritems():
        affectedTeams = []
        for teamkey,team in fts.iteritems():
            if person in team:
                affectedTeams.append(teamkey)
        m.addConstr(sum(x[affectedTeam] for affectedTeam in affectedTeams) == 1)

    for key,person in novices.iteritems():
        affectedTeams = []
        for teamkey,team in fts.iteritems():
            if person in team:
                affectedTeams.append(teamkey)
        m.addConstr(sum(x[affectedTeam] for affectedTeam in affectedTeams) == 1)

    #dummy12 = []
    #for key,team in fts.iteritems():
    #    if "Dummy1" in team and "Dummy2" in team:
    #        dummy12.append(key)
    #m.addConstr(sum(x[team] for team in dummy12) == 1)
   
   
        
    m.modelSense = grb.GRB.MAXIMIZE
    print("---")
    m.optimize()
    print("---")
    count = 0
    for key,value in costs.iteritems():
        if x[key].x != 0:
            try:
                novstring = isAllNov[key]
            except:
                novstring = ""
            print("{}".format(fts[key]))
            count += 1
    print("---")       
    print("{} teams".format(count))


In [12]:
global m
global x
m = grb.Model("Allocate")
x = {}

In [13]:
generatefts(True,True)
pair("Isabella Monardo ","Tara Ma")
allocate()

***Odd number of speakers - Pro Swing added***
156 possible team configurations
---
Optimize a model with 15 rows, 156 columns and 313 nonzeros
Variable types: 0 continuous, 156 integer (156 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 250.0000000
Presolve removed 3 rows and 90 columns
Presolve time: 0.00s
Presolved: 12 rows, 66 columns, 132 nonzeros
Variable types: 0 continuous, 66 integer (66 binary)

Root relaxation: cutoff, 15 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0       250.00000  250.00000  0.00%     -    0s

Explored 0 nodes (15 simplex iterations) in 0.07 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 250 

Optimal solution found (